In [1]:
%run widgets.py

In [8]:
    
s = Segmentation(
    'tls_park', 
    training_repo='/data1/sat/dataset_tls_park_444/train.txt',
    testing_repo='/data1/sat/dataset_tls_park_444/test.txt',
    host='10.10.77.61',
    port=8500,
    #port=8400,
    img_height=480,
    img_width=480,
    model_repo='/data1/sat/xolive/models/model_tls_park_debug',
    nclasses=2,
    #template='deeplab_vgg16',
    iterations=15000,
    test_interval=500,
    snapshot_interval=1000,
    batch_size=5,
    test_batch_size=1,
    noise_prob=0.001,
    class_weights=[0.1, 1.0],
    model_postfix='test_park_xo3',
    weights='/data1/segmentation/vgg16_init_deeplab.caffemodel',
    gpuid=2
)#.run()
s

In [9]:
c = Classification(
    'xo_bbc_5_classes', 
        training_repo='/data1/xolive/bbc_sounds/sounds_dataset_1.0/train/',
        testing_repo='/data1/xolive/bbc_sounds/sounds_dataset_1.0/test/',
                   host='10.10.77.61',
        port=8500,
        model_repo='/data1/bbc_sounds/bbc_5_classes',
        layers='["1CR32", "1CR64", "1CR128", "1024"]',
        template='convnet',
        gpuid=1,
                   img_width=257,
                   img_height=257
        )#.run()
c
# TODO unchanged data

In [2]:
c = Classification('kaggle', 
                    training_repo='/data1/kaggle/dogs_cats/train/',
                   tsplit=.2,
                    host='10.10.77.61',
                    port=8500,
                    model_repo='/data1/xolive/models/kaggle',
                    template='convnet',
                    gpuid=1,
                   layers=['1CR32','1CR64','1CR128','1024'],
                   img_width=150,
                   img_height=150,
                   mirror=True,
                   rotate=True,
        )#.run()
c
# TODO unchanged data

In [60]:
d = Detection(
    "eoliennes",
    training_repo= "/data1/sat/dataset_windturbine_europe_16_test/train.txt",
    testing_repo= "/data1/sat/dataset_windturbine_europe_16_test/test.txt",
    host='10.10.77.61',
    port=8500,
    model_repo='/data1/xolive/models/detection',
    gpuid=0,
    img_width=300,
    img_height=300,
    snapshot_interval=1000,
    iterations=5000,
    template="ssd_300",
    mirror=True,
    rotate=True, # no default
    finetune=True,
    weights="/data1/detection/VGG_ILSVRC_16_layers_fc_reduced.caffemodel",
    batch_size=24,
    test_batch_size=4,
    nclasses=2,
)
d

In [31]:
class CSVTrain(MLWidget):
    
    def __init__(self,
                 sname: str,
                 *args,
                 training_repo: Path = None,
                 model_repo: Path = None,
                 host: str = "localhost",
                 port: int = 1234
                ):
        local_vars = locals()
        params = {
            # no access to eval(k) inside the comprehension
            k: (eval(k, local_vars), v)
            for k, v in get_type_hints(self.__init__).items()
            if k not in ["return", "sname"]
        }

        super().__init__(sname, params)
        
        self._img_explorer = VBox(
            [
                #HBox([HBox([self.train_labels, self.test_labels])]),
                #self.file_list,
                self.output,
            ],
            layout=Layout(width="650px"),
        )

        self._main_elt = HBox(
            [self._configuration, self._img_explorer],
            layout=Layout(width="900px"),
        )
    
    @MLWidget.output.capture(clear_output=True)
    def run(self, *_):
        
        host = self.host.value
        port = self.port.value
    
        body = {
           "mllib":"caffe",
           "description": self.sname,
           "type":"supervised",
           "parameters":{
             "input":{
               "connector":"csv"
             },
             "mllib":{
               "template":"mlp",
               "nclasses":7,
               "layers":[150,150,150],
               "activation":"prelu"
             }
           },
           "model":{
             "templates":"../templates/caffe/",
             "repository": self.model_repo.value
           }
        }
        
        logging.info(
            "Sending request http://{host}:{port}/services/{sname}".format(
                host=host, port=port, sname=self.sname
            )
        )
        c = requests.get(
            "http://{host}:{port}/services/{sname}".format(
                host=host, port=port, sname=self.sname
            )
        )
        logging.info(
            "Current state of service '{sname}': {json}".format(
                sname=self.sname, json=json.dumps(c.json(), indent=2)
            )
        )
        if c.json()["status"]["msg"] != "NotFound":
            self.clear()
            logging.warning(
                (
                    "Since service '{sname}' was still there, "
                    "it has been fully cleared: {json}"
                ).format(sname=self.sname, json=json.dumps(c.json(), indent=2))
            )

        logging.info(
            "Creating service '{sname}':\n {body}".format(
                sname=self.sname, body=json.dumps(body, indent=2)
            )
        )
        c = requests.put(
            "http://{host}:{port}/services/{sname}".format(
                host=host, port=port, sname=self.sname
            ),
            json.dumps(body),
        )

        if c.json()['status']['code'] != 200:
            logging.warning(
                "Reply from creating service '{sname}': {json}".format(
                    sname=self.sname, json=json.dumps(c.json(), indent=2)
                )
            )
        else:
            logging.info(
            "Reply from creating service '{sname}': {json}".format(
                sname=self.sname, json=json.dumps(c.json(), indent=2)
            )
        )
            
        body = {
       "service":"covert",
       "async":True,
       "parameters":{
         "mllib":{
           "gpu":True,
           "solver":{
             "iterations":10000,
             "test_interval":100,
             "base_lr":0.05
           },
           "net":{
             "batch_size":500
           }
         },
         "input":{
         "label_offset":-1,
         "label":"Cover_Type",
         "id":"Id",
         "separator":",",
         "shuffle":True,
         "test_split":0.1,
         "scale":True
         },
         "output":{
           "measure":["acc","mcll","f1"]
         }
       },
       "data":[self.training_repo.value]
     }
        
        logging.info(
            "Start training phase: {body}".format(body=json.dumps(body, indent=2))
        )
        c = requests.post(
            "http://{host}:{port}/train".format(host=host, port=port),
            json.dumps(body),
        )
        logging.info(
            "Reply from training service '{sname}': {json}".format(
                sname=self.sname, json=json.dumps(c.json(), indent=2)
            )
        )

        print(json.dumps(c.json(), indent=2))

In [32]:
CSVTrain('forest',
        host='10.10.77.61',
        port=8500,
        training_repo="/home/xo/forest_type/train.csv",
        model_repo="/data1/xolive/forest/")